In [ ]:
pip install tensorflow scipy requests bluetarget

# Saving Model

To begin with BlueTarget, you will need to save your trained models.

In [ ]:
import tensorflow as tf
print(tf.__version__)


# Creates tensorflow model
model = tf.keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    classifier_activation="softmax",
)

model.save('my_model')

# Creating a service

Services are the core components of BlueTarget, where the serving logic is defined. Create a file name service.py with:

In [ ]:
%%writefile service.py
# service.py
import os
from typing import Dict, List
from scipy.special import softmax
import requests
import tempfile
import numpy as np
import tensorflow as tf

from tensorflow import keras


class Model:

    labels: None

    def __init__(self) -> None:
        self._model = None
        self.labels = requests.get(
            'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
        ).text.split('\n')

    def load(self):
        self._model = keras.models.load_model(
            f"{os.path.dirname(__file__)}/my_model")

    def preprocess(self, image_url: str):
        request = requests.get(image_url)
        with tempfile.NamedTemporaryFile() as f:
            f.write(request.content)
            f.seek(0)
            input_image = tf.image.decode_png(tf.io.read_file(f.name))
        preprocessed_image = tf.keras.applications.resnet_v2.preprocess_input(
            tf.image.resize([input_image], (224, 224))
        )
        return np.array(preprocessed_image)

    def postprocess(self, predictions, k=5):
        class_predictions = predictions[0]
        class_probabilities = softmax(class_predictions)
        top_probability_indices = class_probabilities.argsort()[
            ::-1][:k].tolist()
        return {self.labels[index]: 100 * class_probabilities[index].round(3) for index in top_probability_indices}

    def predict(self, request: Dict) -> Dict[str, List]:

        inputs = request["inputs"]
        predictions = []

        for input in inputs:

            input = self.preprocess(input)
            prediction = self._model.predict(input).tolist()
            prediction = self.postprocess(prediction)
            predictions.append(prediction)

        return {
            "predictions": predictions
        }

# Create requirement file
Here is where you define your model requirements, in our case we have scikit-learn pickle-mixin dependencies

In [ ]:
%%writefile requirements.txt
tensorflow==2.5.0
scipy==1.5.4
requests==2.27.1
numpy==1.19.5

# Create a deployment file

Here you define the deployment configuration, you also can integrate this logic within your deployment pipeline.

You can get your API KEY cliking [here](https://deploy.bluetarget.ai/api-keys)

In [ ]:
from bluetarget import BlueTarget, Server

bt = BlueTarget(api_key="YOUR_API_KEY")

bt.deploy(
    model_class="Model",
    model_files=["service.py", "my_model/saved_model.pb",
                 "my_model/keras_metadata.pb",
                 "my_model/variables/variables.index",
                 "my_model/variables/variables.data-00000-of-00001"],
    requirements_file="requirements.txt",
    serverId=Server.standard_small
)